# Your first scraper

- Repository: `Wikipedia Scrapper`
- Type of Challenge: `Learning`
- Duration: `3 days`
- Submission: `23/06/2023 4:30 PM`

- Team challenge : `Solo`

## The Mission

In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

## Learning objectives

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

* Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.
* Activate it.
* You will find more info about virtual environments in the course content and on the web.
* Note: BEFORE pushing changes into your repo explore what the .gitignore file is and add your environment to it. 

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [4]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = "https://country-leaders.onrender.com/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(f"SUCCESS! We were able to contact the server - Status code 200")
else:
    print(f"FAILED! We were NOT able to contact the server - Status code {req.status_code}")

SUCCESS! We were able to contact the server - Status code 200


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted. The nice thing for us is that JSON files follow a format we already know in Python...dictionaries!

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [5]:
# Set the countries_url variable (1 line)
countries_url = "https://country-leaders.onrender.com/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()

# display the request's status code and the countries variable (1 line)
print(f"STATUS CODE: {req.status_code}, JSON CONTENT: {countries}")

STATUS CODE: 403, JSON CONTENT: {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [6]:
# Set the cookie_url variable (1 line)
cookie_url = "https://country-leaders.onrender.com/cookie/"

# Query the endpoint, set the cookies variable and display it (2 lines)
cookies = requests.get(cookie_url)
cookies = cookies.cookies
print(cookies)

<RequestsCookieJar[<Cookie user_cookie=a81b370a-f3a2-4287-8c8a-14f7463e72a9 for country-leaders.onrender.com/>, <Cookie __cf_bm=ZxLL.hY_JWE07Jv0vDAXVaCXSw971ksj0U6iJYpH2to-1687435037-0-AQBVOsVlvmmuJhxX5vE1Cl6r0WtjmE3a0EK8JzMSIcpDjg1kgTrHN9nqOTkUDesfqQMrkPVonV+Yy+M52gcV+m4= for .onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [7]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries_url = "https://country-leaders.onrender.com/countries"
countries = requests.get(countries_url, cookies=cookies)

# display the countries variable (1 line)
print(countries)
print(type(countries))
for i in countries.json():
    print(i)

<Response [200]>
<class 'requests.models.Response'>
us
ma
be
ru
fr


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [8]:
# Set the leaders_url variable (1 line)
leaders_url = "https://country-leaders.onrender.com/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies)

# display the leaders variable (1 line)
print(leaders)
print(leaders.content)

<Response [404]>
b'{"message":"Please specify a country"}'


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [9]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
params = {'country':'be'}

# assign the output to the leaders variable (1 line)
leaders = requests.get(leaders_url, cookies=cookies, params=params)

# display the leaders variable (1 line)
print(leaders)
print(leaders.content)

<Response [200]>
b'[{"id":"Q12978","first_name":"Guy","last_name":"Verhofstadt","birth_date":"1953-04-11","death_date":null,"place_of_birth":"Dendermonde","wikipedia_url":"https://nl.wikipedia.org/wiki/Guy_Verhofstadt","start_mandate":"1999-07-12","end_mandate":"2008-03-20"},{"id":"Q12981","first_name":"Yves","last_name":"Leterme","birth_date":"1960-10-06","death_date":null,"place_of_birth":"Wervik","wikipedia_url":"https://nl.wikipedia.org/wiki/Yves_Leterme","start_mandate":"2009-11-25","end_mandate":"2011-12-06"},{"id":"Q12983","first_name":"Herman","last_name":"None","birth_date":"1947-10-31","death_date":null,"place_of_birth":"Etterbeek","wikipedia_url":"https://nl.wikipedia.org/wiki/Herman_Van_Rompuy","start_mandate":"2008-12-30","end_mandate":"2009-11-25"},{"id":"Q14989","first_name":"L\xc3\xa9on","last_name":"Delacroix","birth_date":"1867-12-27","death_date":"1929-10-15","place_of_birth":"Saint-Josse-ten-Noode","wikipedia_url":"https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix",

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [10]:
# 4 lines
leaders_per_country = {}
for i in countries.json():
    leaders_per_country[f'{i}'] = requests.get(leaders_url, cookies=cookies, params={"country":f'{i}'}).json()
print(leaders_per_country)
print(type(leaders_per_country))

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

In [11]:
leaders_per_country = {i: requests.get(leaders_url, cookies=cookies, params={"country": i}).json() for i in countries.json()}
print(type(leaders_per_country))
print(leaders_per_country)

<class 'dict'>
{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_dat

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [12]:
# < 15 lines
def get_leaders():
    leaders_url = "https://country-leaders.onrender.com/leaders"
    cookie_url = "https://country-leaders.onrender.com/cookie/"
    countries_url = "https://country-leaders.onrender.com/countries"
    cookies = requests.get(cookie_url)
    cookie_number = cookies.cookies
    countries = requests.get(countries_url, cookies=cookie_number)
    leaders_per_country = {}
    for i in countries.json():
        params = {"country":f'{i}'}
        leaders = requests.get(leaders_url, cookies=cookie_number, params=params)
        leaders_per_country[f'{i}'] = leaders.json()
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [13]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)
print(type(leaders_per_country))

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [14]:
# 3 lines
wiki_url = leaders_per_country["be"][0]["wikipedia_url"]
r = requests.get(wiki_url).text
print(r)

<!DOCTYPE html>
<html class="client-nojs" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Guy Verhofstadt - Wikipedia</title>
<script>document.documentElement.className="client-js";(function(){var cookie=document.cookie.match(/(?:^|; )nlwikimwclientprefs=([^;]+)/);if(cookie){var featureName=cookie[1];document.documentElement.className=document.documentElement.className.replace(featureName+'-enabled',featureName+'-disabled');}}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","januari","februari","maart","april","mei","juni","juli","augustus","september","oktober","november","december"],"wgRequestId":"44b13c21-8341-45a7-a169-50e207721fc8","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Guy_Verhofstadt","wgTitle":"Guy Verhofstadt","wgCurRevisionId":64260130,"wgRevisionId":64260130,"wgArticleId":27801,"wgIs

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [15]:
# 3 lines
from bs4 import BeautifulSoup

soup = BeautifulSoup(r, "html")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Guy Verhofstadt - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";(function(){var cookie=document.cookie.match(/(?:^|; )nlwikimwclientprefs=([^;]+)/);if(cookie){var featureName=cookie[1];document.documentElement.className=document.documentElement.className.replace(featureName+'-enabled',featureName+'-disabled');}}());RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","januari","februari","maart","april","mei","juni","juli","augustus","september","oktober","november","december"],"wgRequestId":"44b13c21-8341-45a7-a169-50e207721fc8","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Guy_Verhofstadt","wgTitle":"Guy Verhofstadt","wgCurRevisionId":64260130,"wgRevisionId":64260130,"wgArt

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [16]:
# 2 lines
paragraphs = []
for elem in soup.find_all("p"):
        paragraphs.append(elem)
paragraphs[0]

<p><b>Guy Maurice Marie-Louise Verhofstadt</b><sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup> (<a href="/wiki/Media:Nl-be_guy_verhofstadt.ogg" title="Geluidsfragment"><img alt="Geluidsfragment" class="noviewer" data-file-height="11" data-file-width="11" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Speakerlink.svg/12px-Speakerlink.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Speakerlink.svg/18px-Speakerlink.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Speakerlink.svg/24px-Speakerlink.svg.png 2x" width="12"/></a> <a class="internal" href="//upload.wikimedia.org/wikipedia/commons/d/dc/Nl-be_guy_verhofstadt.ogg" title="Nl-be guy verhofstadt.ogg">uitspraak</a> <small>(<a href="/wiki/Bestand:Nl-be_guy_verhofstadt.ogg" title="Bestand:Nl-be guy verhofstadt.ogg">info</a> / <a href="/wiki/Help:Ogg_Vorbis" title="Help:Ogg Vorbis">uitleg</a>)</small>) (<a href="/wiki/Dendermonde" title

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [29]:
# <10 lines
first_paragraph = ""
for i in paragraphs:
    if i.find("b"):
        first_paragraph += i.text
    
first_paragraph

'Guy Maurice Marie-Louise Verhofstadt[5] (\xa0uitspraak\xa0(info / uitleg)) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Nu is hij lid van het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.[6]\n'

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [41]:
# 10 lines
from bs4 import BeautifulSoup

def get_first_paragraph(wikipedia_url):
    #   print(wikipedia_url) # keep this for the rest of the notebook
    print(wikipedia_url)
    #   [insert your code]
    r = requests.get(wikipedia_url).text
    soup = BeautifulSoup(r, "html")
    paragraphs = []
    for elem in soup.find_all("p"):
        #filter out Francois Hollande (special banner)
        if elem.find_parent(class_="bandeau-cell"):
            continue
        # filter out empty paragraph in some wiki pages
        if elem.text.strip() != "":
            paragraphs.append(elem.text)
            break
    # return only the first of the paragraphs
    return paragraphs[0]


In [58]:
# Test: 3 lines
get_first_paragraph(leaders_per_country["be"][45]["wikipedia_url"])

https://nl.wikipedia.org/wiki/Walth%C3%A8re_Fr%C3%A8re-Orban


'Hubert Joseph Walthère Frère-Orban (Luik, 24 april 1812 – Brussel, 2 januari 1896) was een Belgisch liberaal politicus die zijn stempel heeft gedrukt op de eerste halve eeuw onafhankelijkheid van België.'

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [51]:
# 3 lines
import re

para = get_first_paragraph(leaders_per_country["fr"][1]["wikipedia_url"])
pattern = r"<.*?>|\(\/.*?\;|\[.{1,2}\]|\n|\(\s.*?\)\)|\[.*?\s.*?\]|\[.*?\s.*?\]|\(/.*?\)|\/\'.*?\/|\xa0"
result = re.sub(pattern, "", para)
print(result)
    # Remove all <>  ==> "<.*?>"
    # Remove all strings starting with (\ and ending with ; ==> "\(\/.*?\;"
    # remove [x] where x can be number or letter ==> "\[.\]"
    # remove \n  ==> "\\n"
    # remove "( uitspraak (info /uitleg))" ==> "\(\s.*?\)\)"
    # remove [fʁɑ̃swa ʔɔlɑ̃d] AND [n 2] ==> "\[.*?\s.*?\]"
    # remove (/ʃaʁl də ɡol/[n 2] Écouter) or (/ʒak ʃiʁak/ Écouter) ==> "\(/.*?\)"
    # remove \xa0  ==> "/\xa0"

https://fr.wikipedia.org/wiki/Nicolas_Sarkozy
Nicolas Sarközy de Nagy-Bocsa, dit Nicolas Sarkozy  Écouter), né le 28 janvier 1955 à Paris (France), est un homme d'État français. Il est président de la République française du 16 mai 2007 au 15 mai 2012.



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [61]:
# 10 lines
from bs4 import BeautifulSoup

def get_first_paragraph(wikipedia_url):
    #   print(wikipedia_url) # keep this for the rest of the notebook
    print(wikipedia_url)
    #   get the wiki text
    r = requests.get(wikipedia_url).text
    soup = BeautifulSoup(r, "html")
    paragraphs = []
    # find all the paragraphs and put them in varibale paragraphs
    for elem in soup.find_all("p"):
        #filter out Francois Hollande (special banner)
        if elem.find_parent(class_="bandeau-cell"):
            continue
        # NEED TO FIND MEDVEDEV exception
        # filter out empty paragraphs
        if elem.text.strip() != "":
            paragraphs.append(elem.text)
            break
    # remove phonetics / tags / endlines
    pattern = r"<.*?>|\(\/.*?\;|\[.{1,2}\]|\n|\(\s.*?\)\)|\[.*?\s.*?\]|\[.*?\s.*?\]|\(/.*?\)|\/\'.*?\/|\xa0"
    first_paragraph = re.sub(pattern, "", paragraphs[0])
    #/ˈhaɪrəm juːˈlɪsiːz/  AND ; /ˈkuːlɪdʒ/;
    return first_paragraph
        
get_first_paragraph(leaders_per_country["fr"][1]["wikipedia_url"])

https://fr.wikipedia.org/wiki/Nicolas_Sarkozy


"Nicolas Sarközy de Nagy-Bocsa, dit Nicolas Sarkozy  Écouter), né le 28 janvier 1955 à Paris (France), est un homme d'État français. Il est président de la République française du 16 mai 2007 au 15 mai 2012."

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [62]:
# < 20 lines


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [63]:
# < 20 lines
def get_leaders():
    leaders_url = "https://country-leaders.onrender.com/leaders"
    cookie_url = "https://country-leaders.onrender.com/cookie/"
    countries_url = "https://country-leaders.onrender.com/countries"
    cookies = requests.get(cookie_url).cookies
    countries = requests.get(countries_url, cookies=cookies)
    leaders_per_country = {}
    for i in countries.json():
        params = {"country":f'{i}'}
        leaders = requests.get(leaders_url, cookies=cookies, params=params)
        leaders_per_country[f'{i}'] = leaders.json()
        
        for leader in leaders_per_country[f"{i}"]:
            leader_url = leader["wikipedia_url"]
            print(get_first_paragraph(leader_url))

In [64]:
# Check the output of your function (2 lines)
print(get_leaders())

https://en.wikipedia.org/wiki/George_Washington
George Washington (February 22, 1732– December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.
https://en.wikipedia.org/wiki/Barack_Obama
Barack Hussein Obama II  born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American  president of the United States. Obama previously served as a U.S. senat

TypeError: string indices must be integers, not 'str'

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [65]:
# < 25 lines
def get_leaders():
    leaders_url = "https://country-leaders.onrender.com/leaders"
    cookie_url = "https://country-leaders.onrender.com/cookie/"
    countries_url = "https://country-leaders.onrender.com/countries"
    cookies = requests.get(cookie_url).cookies

    countries = requests.get(countries_url, cookies=cookies)
    leaders_per_country = {}
    for i in countries.json():
        params = {"country":f'{i}'}
        try:
            leaders = requests.get(leaders_url, cookies=cookies, params=params)
            leaders_per_country[f'{i}'] = leaders.json()
            
            for leader in leaders_per_country[f"{i}"]:
                leader_url = leader["wikipedia_url"]
                print(get_first_paragraph(leader_url))
        except:
            cookies = requests.get(cookie_url).cookies
            continue

Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)
get_leaders()

https://en.wikipedia.org/wiki/George_Washington
George Washington (February 22, 1732 – December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation's founding.[10]

https://en.wikipedia.org/wiki/Barack_Obama
Barack Hussein Obama II  born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American  president of the United States. Obama previously served as a U.S.

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [78]:
# < 20 lines
def get_first_paragraph(wikipedia_url):
    #   print(wikipedia_url) # keep this for the rest of the notebook
    print(wikipedia_url)
    #   get the wiki text
    session = requests.Session()
    r = session.get(wikipedia_url).text
    soup = BeautifulSoup(r, "html")
    paragraphs = []
    # find all the paragraphs and put them in varibale paragraphs
    for elem in soup.find_all("p"):
        # filter out Francois Hollande (special banner)
        if elem.find_parent(class_="bandeau-cell") or elem.find_parent(class_="plainlist"):
            continue
        # filter out empty paragraphs
        if elem.text.strip() != "":
            paragraphs.append(elem.text)
            break
    # remove phonetics / tags / endlines
    pattern = r"<.*?>|\(\/.*?\;|\[.{1,2}\]|\n|\(\s.*?\)\)|\[.*?\s.*?\]|\[.*?\s.*?\]|\(/.*?\)|\/.*?\;|\xa0"
    first_paragraph = re.sub(pattern, "", paragraphs[0])
    return first_paragraph


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [79]:
# <25 lines
def get_leaders():
    leaders_url = "https://country-leaders.onrender.com/leaders"
    cookie_url = "https://country-leaders.onrender.com/cookie/"
    countries_url = "https://country-leaders.onrender.com/countries"
    # start session
    session = requests.Session()
    # request cookie
    cookies = session.get(cookie_url).cookies
    # get countries
    countries = session.get(countries_url, cookies=cookies)
    leaders_per_country = {}
    full_text = []
    # list over all the countries we have grabbed and figure out the leaders link of each country
    for i in countries.json():
        params = {"country":f'{i}'}
        if countries.status_code == 200:
            cookies = session.get(cookie_url).cookies
            leaders = session.get(leaders_url, cookies=cookies, params=params)
            leaders_per_country[f'{i}'] = leaders.json()
            # iterate over the leaders and take the url
            for leader in leaders_per_country[f"{i}"]:
                leader_url = leader["wikipedia_url"]
                # get the first paragraph out of the wiki links
                first_paragraph = get_first_paragraph(leader_url)
                leader["SUMMARY"] = first_paragraph
                full_text.append(first_paragraph)
        else:
            cookies = session.get(cookie_url).cookies
    
    #return full_text and the result (just to compare and see if all characters all OK)
    return full_text, leaders_per_country

Test your new functions.



In [80]:
get_leaders()

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/John_Adams
https://en.wikipedia.org/wiki/Thomas_Jefferson
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
ht

['George Washington (February 22, 1732– December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led Patriot forces to victory in the American Revolutionary War and served as president of the Constitutional Convention of 1787, which created and ratified the Constitution of the United States and the American federal government. Washington has been called the "Father of his Country" for his manifold leadership in the nation\'s founding.',
 'Barack Hussein Obama II  born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African-American  president of the United States. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 199

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines
import json
# Open the file in write mode
with open("leaders.json", "w") as file:
    # Write the JSON object to the file
    json.dump(leaders_per_country, file)
# Close the file when done writing
file.close()


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines
import json
# Open the JSON file in read mode
with open("leaders.json", "r") as file:
    # load the Json data out of the Json file
    data = json.load(file)
#print the Json data
print(data)


{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines
def save(leaders_per_country):
    with open("leaders.json", "w") as file:
    # Write the JSON object to the file
        json.dump(leaders_per_country, file)
    # Close the file when done writing
    file.close()

In [ ]:
# Call the function (1 line)
save(leaders_per_country)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

### Constraints

#### Code style

- Your code should be **commented**. You can use a docstring to explain what your functions are doing. E.g. 

```python
def add(number_one, number_two):
    """
    Function that will perform the add operation between two numbers (in params).
    """
    result = number_one + number_two
    return result
```


- Your code should be **cleaned of any commented, unused code**.

### GitHub Repository

- Push your code on a GitHub repository. The name is at the top of this page.
- Your repository should only contain the files specified.
- Your `README.md` should be nice and complete.
  It should include the following outline:
  - Description (What is the project about? What is the goal? What is your approach?)
  - Installation (What are the requirements to use your code?)
  - Usage (How does someone use your code?)
  - Timeline (How long did the project take?)
  - Personal situation (e.g. This project was done as part of the AI Boocamp at BeCode.org.)

Feel free to add more information.

## Evaluation criteria

| Criteria       | Indicator                                                   | Yes/No |
| -------------- | ----------------------------------------------------------- | ------ |
| 1. Is complete | The student has realized all must-have features.            |        |
|                | There is a published GitHub repo available.                 |        |
|                | The scrapper starts by running `python main.py` in the terminal |        |
| 2. Is correct  | The code is well typed.                                     |        |
|                | All the constraints are respected                           |        |


## To go further (if time permits)

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!

![wikepedia_gif](https://media.giphy.com/media/mADzY6J0QjniQl9gs5/giphy.gif)